## ResNet-50 code below

### Import Dependencies

In [1]:
# pip install tensorflow<2.11
# pip install numpy

import tensorflow as tf
import numpy as np
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


### Splitting the data

In [2]:
# for the directory, please model it as following:
# ebn-images/
# ...A/
# ......a_image_1.jpg
# ......a_image_2.jpg
# ...AA/
# ......b_image_1.jpg
# ......b_image_2.jpg
# ...AAA/
# ......a_image_1.jpg
# ......a_image_2.jpg
# ...AAAA/
# ......b_image_1.jpg
# ......b_image_2.jpg
# ...other/
# ......b_image_1.jpg
# ......b_image_2.jpg


# some splitting of data stuff
img_height, img_width = 256, 256 # must provide, and all images must be the same size
BATCH_SIZE = 32 # should be small if there's not a lot of data
EPOCH = 5

train_ds = tf.keras.utils.image_dataset_from_directory(
    "ebn_images",
    validation_split=0.2,
    subset="training",
    seed=123,
    labels='inferred',
    label_mode='int',
    image_size=(img_height, img_width),
    batch_size=BATCH_SIZE)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    "ebn_images",
    validation_split=0.2,
    subset="validation",
    seed=123,
    labels='inferred',
    label_mode='int',
    image_size=(img_height, img_width),
    batch_size=BATCH_SIZE)

Found 20 files belonging to 5 classes.
Using 16 files for training.
Found 20 files belonging to 5 classes.
Using 4 files for validation.


### Compiling the model

In [3]:
resnet_model = tf.keras.Sequential()
resnet = tf.keras.applications.resnet50.ResNet50(include_top=False, input_shape=(256, 256, 3))
resnet_model.add(resnet)

resnet_model.add(tf.keras.layers.Flatten())

# 128 neurons because our input data is not a lot
resnet_model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
resnet_model.add(tf.keras.layers.Dense(5, activation=tf.nn.softmax))

resnet_model.compile(optimizer=tf.keras.optimizers.SGD(
    learning_rate=1e-1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training the model

In [4]:
resnet_model.fit(train_ds, epochs=EPOCH) 

Epoch 1/5
1/1 [==============================] - 9s 9s/step - loss: 4.0273 - accuracy: 0.0625
Epoch 2/5
1/1 [==============================] - 0s 258ms/step - loss: 444.3055 - accuracy: 0.6875
Epoch 3/5
1/1 [==============================] - 0s 262ms/step - loss: 1.6633 - accuracy: 0.1250
Epoch 4/5
1/1 [==============================] - 0s 260ms/step - loss: 1.3504 - accuracy: 0.8125
Epoch 5/5
1/1 [==============================] - 0s 259ms/step - loss: 1.3185 - accuracy: 0.8125


### Evaluating the Model

In [5]:
# some code of evaluating the model here
test_loss, test_acc = resnet_model.evaluate(validation_ds)

print('Test accuracy:', test_acc)

1/1 [==============================] - 2s 2s/step - loss: 665577650033118933590053805208109056.0000 - accuracy: 0.2500
Test accuracy: 0.25
